In [ ]:
import healpy as hp
import numpy as np
import pickle
import h5py
import altair as alt
alt.renderers.enable('html')
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import pysm3.units as u

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
instrument_model = "simonsobs_instrument_parameters_2023.03/simonsobs_instrument_parameters_2023.03.tbl"

In [ ]:
input_telescope = "SAT"
pol = "T"

In [ ]:
selection = {}
cl = {}
i_pol = "TEB".index(pol)

In [ ]:
if input_telescope.startswith("CH"):
    site = "Chile"
elif input_telescope.startswith("SP"):
    site = "Pole"
else:
    site = "Both"
telescope = input_telescope[-3:]

In [ ]:
cd ..

In [ ]:
from astropy.table import QTable

In [ ]:
chs = QTable.read(instrument_model, format="ascii.ipac")

In [ ]:
has_site = "site" in chs.columns

In [ ]:
chs = chs[chs["telescope"] == telescope]

if has_site:
    chs = chs[chs["site"] == site]

In [ ]:
chs

In [ ]:
alt.data_transformers.enable("default")

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
import pysm3 as pysm
from pysm3 import units as u

In [ ]:
import mapsims

In [ ]:
pol_label = ["TT", "EE", "BB"][i_pol]

In [ ]:
from glob import glob

In [ ]:
toml_files = glob("*.toml")
toml_files.remove("common.toml")
toml_files = [t for t in toml_files if not t.startswith("co") and not t.startswith("radio")]
toml_files.sort()

In [ ]:
pysm_nside = 512 if telescope == "SAT" else 2048

In [ ]:
channels = chs["tag"]

In [ ]:
for config_file in toml_files: 
    mapsim = mapsims.from_config(["common.toml", config_file], override={"channels":"all"})
    component = config_file.split(".")[0]
    print(component)
    filename = f"output/C_ell_{component}.pkl"
    with open(filename, "rb") as f:
        cl[component] = pickle.load(f)
    sky = pysm.Sky(nside=pysm_nside,
                   preset_strings=[mapsim.pysm_components_string],
                   output_unit=u.uK_CMB
                  )
    onecl = pd.DataFrame(
        {ch:pd.Series(cl[component][ch][i_pol]) for ch in channels}
    ).reset_index().rename(columns={"index":"ell"})

    pysm_emission = sky.get_emission(chs["center_frequency"][0])
    label = "PySM " + channels[0]


    pysm_low = hp.anafast(
        pysm_emission,
     lmax=int(1.5*pysm_nside), use_pixel_weights=True)[i_pol]
    onecl[label] = 0
    onecl[label][:len(pysm_low)] = pysm_low

    pysm_emission = sky.get_emission(
                chs[-1]["center_frequency"],
            )
    label = "PySM " + channels[-1]


    pysm_high = hp.anafast(
            pysm_emission,    
        lmax=int(1.5*pysm_nside), use_pixel_weights=True)[i_pol]
    onecl[label] = 0
    onecl[label][:len(pysm_high)] = pysm_high

    onecl = onecl[onecl.ell != 0]
    if telescope == "LAT":
        decim = np.concatenate([[0], np.logspace(0, 4, 1000, endpoint=False)])
        onecl = onecl.iloc[decim,:] # decimate in log scale to make plots smaller

    onecl_long = onecl.melt("ell", var_name="channel")
    onecl_long[onecl_long.value == 0] = np.nan
    selection[component] = alt.selection_multi(fields=['channel'], bind="legend")

    chart = alt.Chart(onecl_long, title=component + " " + pol_label).mark_line().encode(
        x=alt.X("ell", scale=alt.Scale(type='log', base=10, domain=[1,1e4])),
        y=alt.Y("value", scale=alt.Scale(type='log', base=10)),
        color=alt.Color('channel', 
            sort=["PySM " + channels[0]] + list(channels) + ["PySM " + channels[-1]]
                       ),
        opacity=alt.condition(selection[component], alt.value(1), alt.value(0.2))
    ).add_selection(selection[component]).interactive()
    chart.display()
    #chart.save(f"verification/out_plot_cl/{component}_{input_telescope}_{pol}.json")

In [ ]:
# Make sure the notebook is saved

from ipylab import JupyterFrontEnd

app = JupyterFrontEnd()
app.commands.execute('docmanager:save')